In [41]:
from resets import *
from prefutils import *

# initialize web cache
SESSION = init_fetch_session()

## Setup global for now
gwo_months = [3, 6, 9, 12]



In [92]:
# Next Steps
#
# 1. read in CSV table of fixed resets; update prices
# 2. compute the current market spread
# 3. compute future dividend (add to table)
# 4. given a market spread, compute price as a function of various GOC5 estimate
# 5. loop over multiple MS scenarios. Unlike specifying explicitly, use functional guess


resets = pd.read_csv('./resets.csv', dtype={'Notes': object})

# convert dates
resets['ResetDT'] = [csv_to_date(x) for x in resets['Reset Date']]

resets.drop(columns=['Notes','RefPrice','Reset Date'],inplace=True,errors='ignore')

df = resets.copy()
TICKER_BLACKLIST = ['BBD.PR.D', 'GMP.PR.B', 'BIR.PR.A','CSE.PR.A']

# remove unacceptable tickers
df = df[[x not in TICKER_BLACKLIST for x in df['Ticker']]]


df

,Ticker,Rating,Div,Spread,ResetDT
0,AIM.PR.A,ZR,1.1250,375,2020-03-15
1,AIM.PR.C,ZR,1.5025,420,2024-03-15
2,ALA.PR.A,P3I,0.8450,266,2020-09-15
3,ALA.PR.E,P3I,1.2500,317,2023-12-15
4,ALA.PR.G,P3I,1.0605,306,2024-09-15
...,...,...,...,...,...
169,TRP.PR.G,P2L,0.9500,296,2020-11-15
170,TRP.PR.J,P2L,1.3750,469,2021-05-15
171,TRP.PR.K,P2L,1.2250,385,2022-05-15
172,W.PR.K,P2L,1.3125,426,2021-01-15


In [93]:
df = update_data_frame_with_prices_and_drop_reference(df, SESSION)
df

,Ticker,Rating,Div,Spread,ResetDT,Price
0,AIM.PR.A,ZR,1.1250,375,2020-03-15,16.95
1,AIM.PR.C,ZR,1.5025,420,2024-03-15,19.49
2,ALA.PR.A,P3I,0.8450,266,2020-09-15,15.83
3,ALA.PR.E,P3I,1.2500,317,2023-12-15,19.86
4,ALA.PR.G,P3I,1.0605,306,2024-09-15,17.90
...,...,...,...,...,...,...
169,TRP.PR.G,P2L,0.9500,296,2020-11-15,19.05
170,TRP.PR.J,P2L,1.3750,469,2021-05-15,26.06
171,TRP.PR.K,P2L,1.2250,385,2022-05-15,25.33
172,W.PR.K,P2L,1.3125,426,2021-01-15,25.53


In [105]:


foo = update_dataframe_with_market_spread_from_dividend(df, 1.58)
foo = update_dataframe_with_rating_averages(foo)
foo.sort_values(by='AvgSpread',ascending=False).head(50)
#  df = floats_frame[floats_frame["Reference"]=='T'].copy()

# Current yield

,Ticker,Rating,Div,Spread,ResetDT,Price,MSpread,AvgSpread,SpreadToAverage
47,AZP.PR.B,P5H,1.39250,418,2024-12-15,18.40,5.9879,5.987900,0.000000
45,TA.PR.H,P3L,1.25000,365,2022-09-15,17.55,5.5425,4.862014,0.680486
44,TA.PR.F,P3L,1.00675,310,2022-06-15,15.30,5.0001,4.862014,0.138086
33,AX.PR.A,P3L,1.31250,406,2022-09-15,23.07,4.1092,4.862014,-0.752814
34,AX.PR.E,P3L,1.36800,330,2023-09-15,21.72,4.7183,4.862014,-0.143714
35,CF.PR.A,P3L,0.97125,321,2021-09-15,13.95,5.3824,4.862014,0.520386
36,CF.PR.C,P3L,1.25000,403,2022-06-15,17.19,5.6917,4.862014,0.829686
37,CPX.PR.A,P3L,0.76500,217,2020-12-15,13.45,4.1077,4.862014,-0.754314
38,CPX.PR.C,P3L,1.29750,323,2023-12-15,18.82,5.3143,4.862014,0.452286
39,CPX.PR.E,P3L,1.30950,315,2023-06-15,18.07,5.6668,4.862014,0.804786


In [87]:

CURRENT_GOC5_PERCENT = 1.58
GOC5_SCN = { 
    "Constant" :  (1.58,  0.20),
   "SlightDrop": (1.35,  0.30),
   "Drop":       (1.10,  0.30) ,
    "Panic":      (0.80,  0.20)
}

def create_freset_scenarios(df, scenarios, ms_model, enable_hack=False, minspread=MINIMUM_TBILL_MARKET_SPREAD) :
    df["EffMSpread"] = [max(x, minspread) for x in df["MSpread"]]
    
    if enable_hack:
        adjust_hack = df['Ticker'].map(SPREAD_ADJUST).fillna(value=0)
        df['EffMSpread'] += adjust_hack
        
    df["Expected_Gain"] = 0.0

    # for now, ignore 'which model'

    # For each scenario, compute yield to maturity
    today = datetime.datetime.today()
    maturity_date = today+datetime.timedelta(days=365*5)
    month_cycle = gwo_months # GLOBAL!
       
    for scn_name,(futgoc5_percent,probability) in scenarios.items() :
        scenario_ytm = 'YTM'+scn_name
        
        df[scenario_ytm] = [
            compute_ytm_cliff(30*10, # number of days before cliff
                              today ,
                              curprice, # Current price
                              curdiv, # Current div
                              reset_date,
                              reset_spread_bips, # Bips
                              mspread_percent,  # MSpread in Percent
                              futgoc5_percent,   # Future GOC5
                              maturity_date ,
                              month_cycle)
            for (curprice, curdiv, reset_date, reset_spread_bips, mspread_percent)
            in zip(df['Price'],df['Div'],df['ResetDT'],df['Spread'],df['MSpread'])
            ]
                  
        df['Expected_Gain'] += (df[scenario_ytm] * probability)
        
    return df

bar = create_freset_scenarios(foo,GOC5_SCN,"ignore")
bar.head(3)

,Ticker,Rating,Div,Spread,ResetDT,Price,MSpread,EffMSpread,Expected_Gain,YTMConstant,YTMSlightDrop,YTMDrop,YTMPanic
0,AIM.PR.A,ZR,1.1250,375,2020-03-15,16.95,5.0572,5.0572,0.088774,0.081314,0.085541,0.090784,0.098069
1,AIM.PR.C,ZR,1.5025,420,2024-03-15,19.49,6.1291,6.1291,0.071411,0.072299,0.071746,0.071145,0.070422
2,ALA.PR.A,P3I,0.8450,266,2020-09-15,15.83,3.7580,3.7580,0.079168,0.067259,0.074007,0.082368,0.094017


In [90]:
bar.sort_values(by='Expected_Gain',ascending=False).head(30)

,Ticker,Rating,Div,Spread,ResetDT,Price,MSpread,EffMSpread,Expected_Gain,YTMConstant,YTMSlightDrop,YTMDrop,YTMPanic
18,BAM.PR.G,P2L,0.68750,230,2021-11-15,14.46,3.1745,3.1745,0.122416,0.125928,0.123742,0.121361,0.118499
153,TA.PR.D,P3L,0.67725,203,2021-03-15,12.00,4.0637,4.0637,0.120362,0.125976,0.122478,0.118674,0.114108
104,HSE.PR.A,P2L,0.60100,173,2021-03-15,12.11,3.3828,3.3828,0.120193,0.125672,0.122257,0.118545,0.114090
53,CF.PR.A,P3L,0.97125,321,2021-09-15,13.95,5.3824,5.3824,0.116405,0.120644,0.118005,0.115132,0.111677
97,FN.PR.A,P3I,0.69750,207,2021-03-15,12.28,4.1000,4.1000,0.116349,0.121874,0.118431,0.114687,0.110194
166,TRP.PR.C,P2L,0.56575,154,2021-01-15,13.09,2.7420,2.7420,0.113081,0.118132,0.114984,0.111562,0.107454
141,PWF.PR.P,P2H,0.57650,160,2021-01-15,13.95,2.5526,2.5526,0.111137,0.115873,0.112921,0.109713,0.105862
25,BCE.PR.M,P3I,0.69100,209,2021-03-15,14.80,3.0889,3.0889,0.108765,0.113304,0.110475,0.107400,0.103709
114,MFC.PR.F,P2I,0.54450,141,2021-06-15,13.50,2.4533,2.4533,0.108246,0.112772,0.109953,0.106885,0.103200
151,SLF.PR.H,P1L,0.71050,217,2021-09-15,16.89,2.6266,2.6266,0.101435,0.104822,0.102713,0.100416,0.097657


In [81]:
# why is TA.PR.D so high??
ytm = compute_ytm_cliff(30*10, # number of days before cliff
                              today ,
                              12.20, # Current price
                              0.677, # Current div
                              datetime.datetime(2021,3,15),
                              203, # Bips
                              4.8,  # MSpread in Percent
                              1.58,   # Future GOC5
                              datetime.datetime(2025,1,19) ,
                              gwo_months, verbose=True)
print('YTM:', ytm)

MSpread:  4.8
[(datetime.datetime(2020, 1, 19, 18, 42, 38, 43758), -12.2),
 (datetime.datetime(2020, 3, 31, 0, 0), 0.16925),
 (datetime.datetime(2020, 6, 30, 0, 0), 0.16925),
 (datetime.datetime(2020, 9, 30, 0, 0), 0.16925),
 (datetime.datetime(2020, 12, 31, 0, 0), 0.16925),
 (datetime.datetime(2021, 3, 31, 0, 0), 0.225625),
 (datetime.datetime(2021, 6, 30, 0, 0), 0.225625),
 (datetime.datetime(2021, 9, 30, 0, 0), 0.225625),
 (datetime.datetime(2021, 12, 31, 0, 0), 0.225625),
 (datetime.datetime(2022, 3, 31, 0, 0), 0.225625),
 (datetime.datetime(2022, 6, 30, 0, 0), 0.225625),
 (datetime.datetime(2022, 9, 30, 0, 0), 0.225625),
 (datetime.datetime(2022, 12, 31, 0, 0), 0.225625),
 (datetime.datetime(2023, 3, 31, 0, 0), 0.225625),
 (datetime.datetime(2023, 6, 30, 0, 0), 0.225625),
 (datetime.datetime(2023, 9, 30, 0, 0), 0.225625),
 (datetime.datetime(2023, 12, 31, 0, 0), 0.225625),
 (datetime.datetime(2024, 3, 31, 0, 0), 0.225625),
 (datetime.datetime(2024, 6, 30, 0, 0), 0.225625),
 (datet

In [79]:
# if time to reset is less than cliff, use 'anticipated'

# function is passed in the current market spread, but decides whether to use it or not
def compute_ytm_cliff(cliff_days,cur_date,curprice, curdiv, 
                     reset_date, ir_spread_bips,
                    mspread_percent, future_goc5_percent,
                      maturity_date, month_cycle, verbose=False): 
    
    future_div = dividend_after_reset(ir_spread_bips, future_goc5_percent/100)
    
    if ((reset_date-cur_date).days > cliff_days)  :
        # Too far away; use current market spread
        which_mspread_percent = mspread_percent
    else:
        # Anticipated: use future dividend to compute market spread
        # notice the alternate Market Spread is computed using the CURRENT GOC5
        # the assumption is that participants aren't guessing as to the future GOC5.      
        which_mspread_percent = market_spread_from_dividend_in_percent(
                            curprice, future_div, curgoc5_percent)  
    if verbose:
        print("MSpread: ", which_mspread_percent)
    ytm = compute_ytm(today ,
                    curprice, # Current price
                    curdiv, # Current div
                    reset_date,
                    ir_spread_bips,
                    which_mspread_percent,  # MSpread in Percent
                    futgoc5_percent,   # Future GOC5
                    future_div,
                    maturity_date ,
                    month_cycle, verbose)
    return ytm



In [64]:
    
today = datetime.datetime.today()
maturity_date = today+datetime.timedelta(days=365*5)
reset_date = datetime.datetime(2020,6,15)

curprice = 12.2
curdiv = 0.538
curgoc5_percent = 1.58
futgoc5_percent = curgoc5_percent
reset_spread_bips = 128

mspread_percent = market_spread_from_dividend_in_percent(
                curprice, curdiv,curgoc5_percent)

ytm = compute_ytm_cliff(30*6, # number of days before cliff
                        today ,
                       curprice, # Current price
                        curdiv, # Current div
                       reset_date,
                       reset_spread_bips, # Bips
                       mspread_percent,  # MSpread in Percent
                       futgoc5_percent,   # Future GOC5
                       maturity_date ,
                       gwo_months, verbose=True)

print("YTM at 6 month cliff:", ytm)

ytm = compute_ytm_cliff(30*1, # number of days before cliff
                        today ,
                       curprice, # Current price
                        curdiv, # Current div
                       reset_date,
                       reset_spread_bips, # Bips
                       mspread_percent,  # MSpread in Percent
                       futgoc5_percent,   # Future GOC5
                       maturity_date ,
                       gwo_months, verbose=True)

print("YTM at 1 month cliff:", ytm)

[(datetime.datetime(2020, 1, 19, 18, 42, 38, 43758), -12.2),
 (datetime.datetime(2020, 3, 31, 0, 0), 0.1345),
 (datetime.datetime(2020, 6, 30, 0, 0), 0.17875),
 (datetime.datetime(2020, 9, 30, 0, 0), 0.17875),
 (datetime.datetime(2020, 12, 31, 0, 0), 0.17875),
 (datetime.datetime(2021, 3, 31, 0, 0), 0.17875),
 (datetime.datetime(2021, 6, 30, 0, 0), 0.17875),
 (datetime.datetime(2021, 9, 30, 0, 0), 0.17875),
 (datetime.datetime(2021, 12, 31, 0, 0), 0.17875),
 (datetime.datetime(2022, 3, 31, 0, 0), 0.17875),
 (datetime.datetime(2022, 6, 30, 0, 0), 0.17875),
 (datetime.datetime(2022, 9, 30, 0, 0), 0.17875),
 (datetime.datetime(2022, 12, 31, 0, 0), 0.17875),
 (datetime.datetime(2023, 3, 31, 0, 0), 0.17875),
 (datetime.datetime(2023, 6, 30, 0, 0), 0.17875),
 (datetime.datetime(2023, 9, 30, 0, 0), 0.17875),
 (datetime.datetime(2023, 12, 31, 0, 0), 0.17875),
 (datetime.datetime(2024, 3, 31, 0, 0), 0.17875),
 (datetime.datetime(2024, 6, 30, 0, 0), 0.17875),
 (datetime.datetime(2024, 9, 30, 0, 

In [39]:



ytm = compute_ytm_cliff(30*6, # number of days before cliff
                        today ,
                       curprice, # Current price
                        curdiv, # Current div
                       reset_date,
                       reset_spread_bips, # Bips
                       mspread_percent,  # MSpread in Percent
                       futgoc5_percent,   # Future GOC5
                       maturity_date ,
                       gwo_months, verbose=True)


print("YTM (current spread): ", ytm)

# Conclusion: large (3% annual!) difference depending on whether market
# spread is current or anticipated. A difference of 0.6% changes the 
# expected share price a LOT.  

# Run across 3 predictive MSpread models:
# Model C6: current if 6mo, else anticipated (calculated as of today)
# Model A: anticipated (use future div in spread calc)

mspread_percent_alt = market_spread_from_dividend_in_percent(
                        curprice, futdiv, curgoc5_percent)

futprice2, futdiv2, ytm2 = compute_ytm(today ,
                       curprice, # Current price
                        curdiv, # Current div
                       reset_date,
                       128, # Bips
                       mspread_percent_alt,  # MSpread in Percent
                       futgoc5_percent,   # Future GOC5
                       maturity_date ,
                       gwo_months)
print("-----")
print ("MSpread % A: ", mspread_percent_alt)
print("Future Price A: " ,futprice2)
print("Future Div A: ", futdiv2)
print("YTM (current spread) A: ", ytm2)

#---------
mspread_percent_mean = 3.4
futprice3, futdiv3, ytm3 = compute_ytm(today ,
                       curprice, # Current price
                        curdiv, # Current div
                       reset_date,
                       128, # Bips
                       mspread_mean_percent_mean,  # MSpread in Percent
                       futgoc5_percent,   # Future GOC5
                       maturity_date ,
                       gwo_months)
print("-----")
print ("MSpread % M: ", mspread_percent_mean)
print("Future Price M: " ,futprice3)
print("Future Div M: ", futdiv3)
print("YTM (current spread) M: ", ytm3)

MSpread %:  2.8298360655737707
Future Div:  0.715
Future Price:  16.21375464684015
YTM (current spread):  0.11293835811628382
-----
MSpread % A:  4.280655737704918
Future Price A:  12.200000000000001
Future Div A:  0.715
YTM (current spread) A:  0.05922933116543647
-----
MSpread % M:  3.4
Future Price M:  14.357429718875503
Future Div M:  0.715
YTM (current spread) M:  0.08948204406012335
